<a href="https://colab.research.google.with/github/maurorisonho/fraud-detection-neuromorphic/blob/main/notebooks/01-stdp_example.ipynb" target="_parent"><img src="https://colab.research.google.with/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STDP Example: Biological Learning

**Description:** Interactive Tutorial about the biological learning mechanism STDP (Spike-Timing-Dependent Plasticity) used in neuromorphic neural networks. Demonstrates how neurons learn temporal correlations automatically.

**Author:** Mauro Risonho de Paula Assumpção.
**Creation Date:** 5 December 2025.
**License:** MIT License.
**Development:** Human + AI Assisted Development (Claude Sonnet 4.5, Gemini 3 Pro Preview).

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
# Install the library Brian2 if not yet installed
try:
    import brian2
except ImportError:
    !pip install brian2
    import brian2

# Specific import of the brian2 instead of wildcard
from brian2 import (
    ms, mV, Hz, second,
    NeuronGroup, Synapses, SpikeMonitor, StateMonitor,
    SpikeGeneratorGroup, Network,
    defaultclock, run, device, start_scope,
    clip, prefs
)

# Configure to use numpy (avoids error of compilation C++ if headers are missing)
prefs.codegen.target = "numpy"

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

print(" Imports completed!")

# STDP: Spike-Timing-Dependent Plasticity

**Description:** Interactive Tutorial about the biological learning mechanism STDP (Spike-Timing-Dependent Plasticity) used in neuromorphic neural networks. Demonstrates how neurons learn temporal correlations automatically.

**Author:** Mauro Risonho de Paula Assumpção
**Creation Date:** 5 December 2025
**License:** MIT License
**Development:** Human + AI-Assisted Development (Claude Sonnet 4.5, Gemini 3 Pro Preview).

---

This notebook explores the biological learning mechanism **STDP** used in neuromorphic neural networks.

## What is STDP?

STDP (Spike-Timing-Dependent Plasticity) is a learning rule **unsupervised** inspired by biological neurons:

- **if the pre-synaptic neuron fires BEFORE the post-synaptic** → **Potentiation** (weight ↑)
- **if the pre-synaptic neuron fires AFTER the post-synaptic** → **Depression** (weight ↓)

This allows the network to learn **temporal causal relationships** without explicit labels.

## 1. Setup and Imports

## 1. Classic STDP Curve

Visualize how the change in weight depends on the temporal difference between spikes.

In [ ]:
# STDP Parameters
tau_pre = 20.0 # ms - time constant pre-synaptic
tau_post = 20.0 # ms - time constant post-synaptic
A_pre = 0.01 # Potentiation Amplitude
A_post = -0.012 # Depression Amplitude

# Delta t (temporal difference)
dt_range = np.linspace(-100, 100, 500) # ms

# Calculate mudança of weight
def stdp_weight_change(dt, tau_pre, tau_post, A_pre, A_post):
    """
    Calculates mudança of weight according to STDP.
    dt = t_post - t_pre
    """
    if dt > 0: # Post after Pre → Potentiation
        return A_pre * np.exp(-dt / tau_pre)
    else: # Post before Pre → Depression
        return A_post * np.exp(dt / tau_post)

weight_changes = np.array([stdp_weight_change(dt, tau_pre, tau_post, A_pre, A_post)
                           for dt in dt_range])

# Plot curve STDP
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(dt_range, weight_changes, linewidth=3, color='purple')
ax.axhline(0, color='black', linestyle='--', linewidth=1, alpha=0.5)
ax.axvline(0, color='black', linestyle='--', linewidth=1, alpha=0.5)

# Anotar regions
ax.fill_between(dt_range[dt_range > 0], 0, weight_changes[dt_range > 0],
                alpha=0.2, color='green', label='Potentiation (LTP)')
ax.fill_between(dt_range[dt_range < 0], 0, weight_changes[dt_range < 0],
                alpha=0.2, color='red', label='Depression (LTD)')

ax.set_xlabel('Δt = t_post - t_pre (ms)', fontsize=12)
ax.set_ylabel('Mudança of weight (Δw)', fontsize=12)
ax.set_title('curve STDP: Spike-Timing-Dependent Plasticity', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Add annotations
ax.annotate('Pré → Pós\n(Causal)', xy=(20, 0.008), fontsize=10,
            ha='center', color='green', fontweight='bold')
ax.annotate('Pós → Pré\n(Anti-causal)', xy=(-20, -0.009), fontsize=10,
            ha='center', color='red', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n Interpretation:")
print(" - Δt > 0: neuron pré fires BEFORE → Potentiation (reforça connection)")
print(" - Δt < 0: neuron pré fires AFTER → Depression (enfraquece connection)")
print(" - Efeito decai exponencialmente with |Δt|")

## 2. Simulation STDP with Brian2

Simular dois neurons conectados with STDP e observar evolution of the weights.

In [ ]:
start_scope()

# Parameters of the Simulation
duration = 100*ms  # type: ignore[operator]
defaultclock.dt = 0.1*ms  # type: ignore[operator]

print(" Configurando Simulation STDP...")
print(f"Duration: {duration}")
print(f"Time step: {defaultclock.dt}\n")

# neurons LIF
tau_m = 10*ms  # type: ignore[operator]
tau_syn = 5*ms  # type: ignore[operator] # Constante of time of the sinapse
v_rest = -70*mV
v_thresh = -50*mV
v_reset = -70*mV

# Adicionado decaimento sináptico (dI_syn/dt)
eqs_post = '''
dv/dt = (v_rest - v + I_syn) / tau_m : volt
dI_syn/dt = -I_syn / tau_syn : volt
'''

# Create neurons
neuron_pre = SpikeGeneratorGroup(1, [0], [10]*ms)  # type: ignore[operator]
neuron_post = NeuronGroup(1, eqs_post, threshold='v > v_thresh',
                          reset='v = v_reset', method='euler')
neuron_post.v = v_rest
neuron_post.I_syn = 0*mV

# STDP Parameters
tau_pre_stdp = 20*ms  # type: ignore[operator]
tau_post_stdp = 20*ms  # type: ignore[operator]
A_pre_stdp = 0.01
A_post_stdp = -0.012
w_max = 1.0
w_min = 0.0

synapse_model = '''
w : 1
dApre/dt = -Apre / tau_pre_stdp : 1 (event-driven)
dApost/dt = -Apost / tau_post_stdp : 1 (event-driven)
'''

# Aumentado ganho sináptico for garantir disparo (w * 60*mV)
on_pre_stdp = '''
I_syn_post += w * 60 * mV
Apre += A_pre_stdp
w = clip(w + Apost, w_min, w_max)
'''

on_post_stdp = '''
Apost += A_post_stdp
w = clip(w + Apre, w_min, w_max)
'''

synapse = Synapses(neuron_pre, neuron_post,
                   model=synapse_model,
                   on_pre=on_pre_stdp,
                   on_post=on_post_stdp,
                   method='euler')
synapse.connect(i=0, j=0)
synapse.w = 0.5 # weight inicial

# Monitores
mon_pre = SpikeMonitor(neuron_pre)
mon_post = SpikeMonitor(neuron_post)
mon_weight = StateMonitor(synapse, 'w', record=True)
mon_voltage = StateMonitor(neuron_post, 'v', record=True)

# Execute Simulation
print("⏳ Executing Simulation Brian2...")
import time
start_time = time.time()

net = Network(neuron_pre, neuron_post, synapse, mon_pre, mon_post, mon_weight, mon_voltage)
net.run(duration)

sim_time = time.time() - start_time

print(f" Simulation concluída in {sim_time:.3f}s!")
print(f"\n Results:")
print(f" Spikes pré-sinápticos: {len(mon_pre.t)}")
print(f" Spikes post-synaptics: {len(mon_post.t)}")
print(f" weight inicial: {0.5:.3f}")
print(f" weight final: {mon_weight.w[0][-1]:.3f}")
print(f" Mudança: {(mon_weight.w[0][-1] - 0.5):.3f} ({(mon_weight.w[0][-1] - 0.5)/0.5*100:+.1f}%)")

In [ ]:
# Visualize results
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Plot 1: Spikes
if len(mon_pre.t) > 0:
    axes[0].eventplot([mon_pre.t/ms], lineoffsets=1, linelengths=0.8,
                      linewidths=2, colors='blue', label='pre-synaptic')
if len(mon_post.t) > 0:
    axes[0].eventplot([mon_post.t/ms], lineoffsets=0, linelengths=0.8,
                      linewidths=2, colors='red', label='post-synaptic')

axes[0].set_ylabel('neuron')
axes[0].set_yticks([0, 1])
axes[0].set_yticklabels(['Pós', 'Pré'])
axes[0].set_title('Raster Plot: Spikes Pré e Pós-Synaptics', fontsize=12, fontweight='bold')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

# Plot 2: Evolution of the Synaptic weight
axes[1].plot(mon_weight.t/ms, mon_weight.w[0], linewidth=2.5, color='purple')
axes[1].axhline(0.5, color='gray', linestyle='--', alpha=0.5, label='weight Inicial')
axes[1].set_ylabel('weight Synaptic (w)')
axes[1].set_title('Evolution of the Synaptic weight with STDP', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot 3: Voltagem of the neuron post-synaptic
axes[2].plot(mon_voltage.t/ms, mon_voltage.v[0]/mV, linewidth=1.5, color='green')
axes[2].axhline(-50, color='red', linestyle='--', alpha=0.7, label='Threshold')
axes[2].axhline(-70, color='gray', linestyle='--', alpha=0.5, label='Resting')
axes[2].set_xlabel('time (ms)')
axes[2].set_ylabel('Voltagem (mV)')
axes[2].set_title('Potencial of Membrana post-synaptic', fontsize=12, fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. STDP with patterns of input

Demonstrate how STDP learns temporal correlations in patterns repeated.

In [ ]:
start_scope()

# Simular multiple neurons pré-sinápticos
n_pre = 5
n_post = 1
duration = 500*ms  # type: ignore[operator]
defaultclock.dt = 0.1*ms  # type: ignore[operator]

print(" Configurando Simulation with multiple neurons...")
print(f"neurons pré-sinápticos: {n_pre}")
print(f"Duration: {duration}\n")

# Generate temporal pattern (some neurons fire in sequence)
spike_pattern = [
    [10, 110, 210, 310, 410], # neuron 0: spikes regulares
    [15, 115, 215, 315, 415], # neuron 1: levemente atrasado
    [20, 120, 220, 320, 420], # neuron 2: more atrasado
    [100, 200, 300, 400], # neuron 3: spikes esparsos
    [50, 150, 250, 350, 450] # neuron 4: fase different
]

indices = []
times = []
print(" patterns of spikes:")
for neuron_idx, spike_times in enumerate(spike_pattern):
    print(f" neuron {neuron_idx}: {len(spike_times)} spikes")
    for t in spike_times:
        indices.append(neuron_idx)
        times.append(t)

neuron_pre = SpikeGeneratorGroup(n_pre, indices, times*ms)  # type: ignore[operator]

# neuron post-synaptic
neuron_post = NeuronGroup(n_post, eqs_post, threshold='v > v_thresh',
                          reset='v = v_reset', method='euler')
neuron_post.v = v_rest

# Sinapses with STDP
synapse = Synapses(neuron_pre, neuron_post,
                   model=synapse_model,
                   on_pre=on_pre_stdp,
                   on_post=on_post_stdp,
                   method='euler')
synapse.connect() # Conectar all
synapse.w = 'rand() * 0.3 + 0.2' # weights initial aleatórios [0.2, 0.5]

# Monitores
mon_pre = SpikeMonitor(neuron_pre)
mon_post = SpikeMonitor(neuron_post)
mon_weight = StateMonitor(synapse, 'w', record=True)

# Save weights initial
initial_weights = np.array(synapse.w).copy()

# Execute
print("\n⏳ Executing Simulation of temporal patterns...")
start_time = time.time()

net = Network(neuron_pre, neuron_post, synapse, mon_pre, mon_post, mon_weight)
net.run(duration)

sim_time = time.time() - start_time
final_weights = np.array(synapse.w).copy()

print(f" Simulation concluída in {sim_time:.3f}s!")
print(f"\n Analysis of the weights Synaptics:")
for i in range(n_pre):
    delta = final_weights[i] - initial_weights[i]
    percentage = (delta / initial_weights[i]) * 100 if initial_weights[i] > 0 else 0
    print(f" neuron {i}: {initial_weights[i]:.3f} → {final_weights[i]:.3f} (Δ = {delta:+.3f}, {percentage:+.1f}%)")

In [ ]:
# Visualize evolution of the weights
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Temporal evolution of the weights
for i in range(n_pre):
 axes[0].plot(mon_weight.t/ms, mon_weight.w[i], label=f'Synapse {i}', linewidth=2)

axes[0].set_xlabel('time (ms)')
axes[0].set_ylabel('weight Synaptic')
axes[0].set_title('Temporal Evolution of Synaptic Weights with STDP', fontsize=12, fontweight='bold')
axes[0].legend(loc='best')
axes[0].grid(True, alpha=0.3)

# Plot 2: Comparison before/after
x_pos = np.arange(n_pre)
width = 0.35

axes[1].bar(x_pos - width/2, initial_weights, width, label='Inicial', alpha=0.7, color='lightblue')
axes[1].bar(x_pos + width/2, final_weights, width, label='Final', alpha=0.7, color='darkblue')

axes[1].set_xlabel('neuron pre-synaptic')
axes[1].set_ylabel('weight Synaptic')
axes[1].set_title('Comparison: weights Initial vs Finais', fontsize=12, fontweight='bold')
axes[1].set_xticks(x_pos)
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n Interpretation:")
print(" - neurons that fire consistentemente BEFORE of the post-synaptic are reinforced")
print(" - neurons with timing inconsistente têm weights reduzidos")
print(" - A network learns a temporal correlation automatically!")

## 4. Application to the Detection of Fraude

how STDP ajuda in the detection of fraude?

### Scenario 1: Normal Temporal Sequence

**Transaction Legítima:**
1. Login in the app (t=0ms)
2. Navigation in the saldo (t=500ms)
3. Selection of beneficiary conhecido (t=2000ms)
4. Payment confirmation (t=3000ms)

**STDP learns:**
- Sequence causal esperada
- Temporal intervals normais
- Reforça connections that represent behavior legítimo

### Scenario 2: Anomalous Sequence (Fraude)

**Transaction Fraudulenta:**
1. Login in the app (t=0ms)
2. Transfer imediata without navigation (t=50ms)
3. high value for new beneficiary (t=100ms)
4. Location geográfica inconsistente (t=150ms)

**STDP detects:**
- temporal pattern anomalous
- Sequence not reinforced during Training
- high activation of neurons of "fraude"

### Vantagens of the STDP:

1. **Learning unsupervised**: not needs of labels explícitos inicialmente
2. **Continuous adaptation**: Learns new fraud patterns automatically
3. **Temporal sensitivity**: Detects anomalies in the sequence of events
4. **efficiency**: Local weight update (without backpropagation)
5. **Biologically plausible**: Inspirado in the cérebro Human

## 5. Conclusões

### STDP in Detection of Fraude

**mechanism:**
- Learns temporal correlations between features of transaction
- Reforça patterns legítimos frequentes
- Detects deviations in the temporal sequence

**Applications Práticas:**
1. **Analysis of behavior**: Sequence of actions in the mobile banking
2. **Detection of speed**: Transactions impossible (ex: compras in cidades different in few minutes)
3. **patterns of usage**: Horários, frequency, values típicos
4. **Suspicious navigation**: Atypical page sequences

**Comparison with methods Traditional:**

| feature | STDP/SNN | DNN/LSTM |
|----------------|----------|----------|
| Temporal processing | Nativo | Emulado |
| Supervisão | not | yes |
| latency | Ultra-low (~ms) | high (~100ms) |
| consumption energético | very low | high |
| Online adaptation | yes | Difficult |
| Hardware especializado | yes (Loihi, TrueNorth) | GPU |

### Futuro

- Chips neuromórficos dedicados (Intel Loihi 2, IBM NorthPole)
- STDP + Reward modulation (dopamina artificial)
- Learning federado with STDP
- Explicabilidade: Visualize weights learned

---

**Author:** Mauro Risonho de Paula Assumpção
**Projeto:** Computation Neuromórfica for Cybersecurity Banking